In [3]:
import numpy as np
from scipy.optimize import least_squares

# Simulated 3D points
n_points = 10
points_3d = np.random.randn(n_points, 3)  # 10 3D points

# Simulated camera parameters (rotation and translation)
R = np.eye(3)  # Identity matrix as a simple rotation
T = np.zeros(3)  # Zero translation
K = np.array([[800, 0, 320],  # Intrinsic matrix (fx, fy, cx, cy)
              [0, 800, 240],
              [0, 0, 1]])

# 2D projections of the 3D points (simulated)
# Here we simulate projections by simply projecting points using the intrinsic matrix and camera pose
def project_points(points, R, T, K):
    # Project 3D points to 2D using the camera parameters
    points_homogeneous = np.hstack([points, np.ones((points.shape[0], 1))])
    print(points_homogeneous.shape)
    print(R.shape)
    projections = (K @ (R @ points_homogeneous.T + T[:, None])).T
    projections = projections[:, :2] / projections[:, 2][:, None]  # Normalize by depth (z)
    return projections

# Get initial 2D projections
initial_2d_points = project_points(points_3d, R, T, K)

# Define the residual function (difference between observed and projected 2D points)
def residuals(params, n_points, K, initial_2d_points):
    R = params[:9].reshape(3, 3)  # First 9 params are rotation (3x3 matrix)
    T = params[9:12]  # Next 3 are translation (3 elements)
    points_3d = params[12:].reshape(n_points, 3)  # Remaining are the 3D points

    projections = project_points(points_3d, R, T, K)  # Project 3D points to 2D
    residual = (projections - initial_2d_points).flatten()  # Flatten the residuals
    return residual

# Initial guess for optimization: R, T, and 3D points (all zeros)
initial_guess = np.hstack([np.eye(3).flatten(), np.zeros(12), points_3d.flatten()])

# Perform optimization (bundle adjustment)
result = least_squares(residuals, initial_guess, args=(n_points, K, initial_2d_points))

# Extract optimized parameters
R_opt = result.x[:9].reshape(3, 3)
T_opt = result.x[9:12]
points_3d_opt = result.x[12:].reshape(n_points, 3)

print("Optimized Rotation:\n", R_opt)
print("Optimized Translation:\n", T_opt)
print("Optimized 3D Points:\n", points_3d_opt)


(10, 4)
(3, 3)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)